# Crawling
- 링크를 전부 가져와서 또 다시 링크로 들어가서 확인해 보는 것
- __링크를 저장__
- q: query

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
headers = {
    "user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36"
}
params = {
    "q" :"한글"
}
url = "http://www.google.com/search"

In [3]:
html = requests.get(url, params=params, headers=headers)

In [4]:
type(html)

requests.models.Response

In [5]:
html.encoding

'UTF-8'

In [6]:
type(html.text)

str

In [7]:
dom = BeautifulSoup(html.text, "lxml")

In [8]:
type(dom) #최상위 노드 

bs4.BeautifulSoup

In [9]:
aList = dom.select('div.rc > h3.r > a')

In [10]:
len(aList)

8

In [11]:
for row in aList:
    print(row.text)

다운로드 : 한글과컴퓨터 - Hancom
Windows : 한글과컴퓨터 - Hancom
한글 - 나무위키
한컴오피스 한글 - 나무위키
한글 - 위키백과, 우리 모두의 백과사전
한국어 입문/한글 자모 - 위키책
한글의 구성 - 알고 싶은 한글
제3회 한글 창의 아이디어 공모전(2017) 공모 < 공지사항<알림마당 ...


In [12]:
for row in aList:
    print(row["href"])

http://www.hancom.com/downLoad.downPU.do
https://www.hancom.com/product/productWindowsMain.do
https://namu.wiki/w/%ED%95%9C%EA%B8%80
https://namu.wiki/w/%ED%95%9C%EC%BB%B4%EC%98%A4%ED%94%BC%EC%8A%A4%20%ED%95%9C%EA%B8%80
https://ko.wikipedia.org/wiki/%ED%95%9C%EA%B8%80
https://ko.wikibooks.org/wiki/%ED%95%9C%EA%B5%AD%EC%96%B4_%EC%9E%85%EB%AC%B8/%ED%95%9C%EA%B8%80_%EC%9E%90%EB%AA%A8
https://www.korean.go.kr/hangeul/principle/001.html
http://www.kocca.kr/cop/bbs/view/B0000137/1833364.do?menuNo=200827&noticevent=Y


In [13]:
def getURL(url, params=None, num_retries=2):
    html = requests.get(url, params=params, headers=headers)
    if 500 <= html.status_code < 600 and nun_retries > 0 :
        print(html.status_code, html.reason)
        return getURL(url, num_retries - 1)
    
    dom = BeautifulSoup(html.text,"lxml")
    links = dom.select("div.rc > h3.r > a") #css selector는 구글에서만 실행
    
    return [link["href"] for link in links]

In [14]:
links = getURL(url, params=params)

In [15]:
links

['http://www.hancom.com/downLoad.downPU.do',
 'https://www.hancom.com/product/productWindowsMain.do',
 'https://namu.wiki/w/%ED%95%9C%EA%B8%80',
 'https://namu.wiki/w/%ED%95%9C%EC%BB%B4%EC%98%A4%ED%94%BC%EC%8A%A4%20%ED%95%9C%EA%B8%80',
 'https://ko.wikipedia.org/wiki/%ED%95%9C%EA%B8%80',
 'https://ko.wikibooks.org/wiki/%ED%95%9C%EA%B5%AD%EC%96%B4_%EC%9E%85%EB%AC%B8/%ED%95%9C%EA%B8%80_%EC%9E%90%EB%AA%A8',
 'https://www.korean.go.kr/hangeul/principle/001.html',
 'http://www.kocca.kr/cop/bbs/view/B0000137/1833364.do?menuNo=200827&noticevent=Y']

In [16]:
def getURL(url, params=None, select="a", num_retries=2):
    html = requests.get(url, params=params, headers=headers)
    
    if 500 <= html.status_code < 600 and nun_retries > 0 :
        print(html.status_code, html.reason)
        return getURL(url, params, select, num_retries - 1)
    
    dom = BeautifulSoup(html.text,"lxml")
    links = dom.select(select) #css selector는 구글에서만 실행
    
    return [link["href"] for link in links if link.has_attr("href")==True]

In [17]:
seed = "http://www.google.com/search"
select = "div.rc > h3.r > a"

queue = getURL(seed, params, select)
queue = [seed]
print(queue)

['http://www.google.com/search']


### 링크 안으로 들어가서 계속 crawling 하는 방법
- queue안에는 들어갔을 때 그 안에 존재하는 링크 태그를 전부 저장한 것이다. 

In [18]:
while queue: #queue 안에 url 있어야 실행
    url = queue.pop()
    queue.extend(getURL(url, params=params, select=select))
    print(url, len(queue))
    select = "a"

http://www.google.com/search 8
http://www.kocca.kr/cop/bbs/view/B0000137/1833364.do?menuNo=200827&noticevent=Y 271


MissingSchema: Invalid URL '#': No schema supplied. Perhaps you meant http://#?

In [19]:
len(queue)

270

- 처음 select를 통해서 구글의 링크를 확인한다.
- 구글의 링크를 확인한 다음에 결과에 저장한다. 
- 처음 페이지 구글 페이지에 존재하는 리스트만 확인한다. 

In [20]:
seed = "http://www.google.com/search"
select = "div.rc > h3.r > a"

queue = getURL(seed, params, select)
result = list()

In [21]:
while queue: #queue 안에 url 있어야 실행
    url = queue.pop()
    result.extend(getURL(url, params=params, select=select))
    print(url, len(result))

http://www.kocca.kr/cop/bbs/view/B0000137/1833364.do?menuNo=200827&noticevent=Y 0
https://www.korean.go.kr/hangeul/principle/001.html 0
https://ko.wikibooks.org/wiki/%ED%95%9C%EA%B5%AD%EC%96%B4_%EC%9E%85%EB%AC%B8/%ED%95%9C%EA%B8%80_%EC%9E%90%EB%AA%A8 0
https://ko.wikipedia.org/wiki/%ED%95%9C%EA%B8%80 0
https://namu.wiki/w/%ED%95%9C%EC%BB%B4%EC%98%A4%ED%94%BC%EC%8A%A4%20%ED%95%9C%EA%B8%80 0
https://namu.wiki/w/%ED%95%9C%EA%B8%80 0
https://www.hancom.com/product/productWindowsMain.do 0
http://www.hancom.com/downLoad.downPU.do 0


# 뽐뿌 자유게시판 실습
- 뽐뿌는 lxml로는 parsing이 되지 않는다.
- accept-encoding을 false로 해야한다. 

In [22]:
def getURL(url, params=None, select='a', num_retries=2):
    html = requests.get(url, params=params, headers=headers,)
    
    if 500 <= html.status_code < 600 and nun_retries > 0 :
        print(html.status_code, html.reason)
        return getURL(url, params, num_retries - 1)
    
    html = html.content.decode('euc-kr').encode('utf-8').decode('utf-8')
    dom = BeautifulSoup(html,"html.parser")
    links = dom.select(select) #css selector는 구글에서만 실행
    return [link['href'] for link in links if link.has_attr('href') == True]

In [23]:
seed = "http://www.ppomppu.co.kr/zboard/zboard.php"
params = {
    "id":"freeboard"
}
headers = {
    "user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36",
    "accept-encoding":"false"
}

html = requests.get(seed, params=params, headers=headers)
html = html.content.decode('euc-kr').encode('utf-8').decode('utf-8')

### lxml은 훨씬 적은 결과가 나온다. 그러므로 잘 안된다는 것을 알 수 있다.
- 스피드적인 부분에서는 좋지만 결과값이 다 나오지는 않는다.

In [24]:
dom = BeautifulSoup(html,"lxml")
dom.prettify()
links = dom.select('a')
#print(dom)
[link['href'] for link in links if link.has_attr('href') == True]

['http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=event',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=buy',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=help',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=etc_info',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=free_picture',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=news2',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=review',
 'http://www.ppomppu.co.kr/recent_main_article.php?type=market',
 'http://www.ppomppu.co.kr/myinfo/env.php?cmd=env',
 'http://www.ppomppu.co.kr/myinfo/member_bookmark.php',
 'http://www.ppomppu.co.kr/index.php',
 '/zboard/zboard.php?id=freeboard',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu2',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu4',


In [25]:
dom = BeautifulSoup(html,"html.parser")
dom.prettify()
links = dom.select('a')
#print(dom)
[link['href'] for link in links if link.has_attr('href') == True]

['http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=event',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=buy',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=help',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=etc_info',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=free_picture',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=news2',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=review',
 'http://www.ppomppu.co.kr/recent_main_article.php?type=market',
 'http://www.ppomppu.co.kr/myinfo/env.php?cmd=env',
 'http://www.ppomppu.co.kr/myinfo/member_bookmark.php',
 'http://www.ppomppu.co.kr/index.php',
 '/zboard/zboard.php?id=freeboard',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu2',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu4',


In [26]:
result = getURL(seed, params)

In [27]:
result

['http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=event',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=buy',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=help',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=etc_info',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=free_picture',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=news2',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=review',
 'http://www.ppomppu.co.kr/recent_main_article.php?type=market',
 'http://www.ppomppu.co.kr/myinfo/env.php?cmd=env',
 'http://www.ppomppu.co.kr/myinfo/member_bookmark.php',
 'http://www.ppomppu.co.kr/index.php',
 '/zboard/zboard.php?id=freeboard',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu2',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu4',


### css selector 변화
- td라는 태그의 class = "list_vspace"이면서 img 옆에 있는 a태그
- select를 변화 시킬 수 있다. 

In [28]:
contentList = getURL(seed, params, "td.list_vspace img + a")

In [29]:
contentList = contentList[1:]

In [30]:
contentList

['view.php?id=freeboard&page=1&divpage=1111&no=5951939',
 'view.php?id=freeboard&page=1&divpage=1111&no=5951938',
 'view.php?id=freeboard&page=1&divpage=1111&no=5951937',
 'view.php?id=freeboard&page=1&divpage=1111&no=5951936',
 'view.php?id=freeboard&page=1&divpage=1111&no=5951935',
 'view.php?id=freeboard&page=1&divpage=1111&no=5951934',
 'view.php?id=freeboard&page=1&divpage=1111&no=5951933',
 'view.php?id=freeboard&page=1&divpage=1111&no=5951932',
 'view.php?id=freeboard&page=1&divpage=1111&no=5951931',
 'view.php?id=freeboard&page=1&divpage=1111&no=5951930',
 'view.php?id=freeboard&page=1&divpage=1111&no=5951929',
 'view.php?id=freeboard&page=1&divpage=1111&no=5951928',
 'view.php?id=freeboard&page=1&divpage=1111&no=5951926',
 'view.php?id=freeboard&page=1&divpage=1111&no=5951925',
 'view.php?id=freeboard&page=1&divpage=1111&no=5951924',
 'view.php?id=freeboard&page=1&divpage=1111&no=5951923',
 'view.php?id=freeboard&page=1&divpage=1111&no=5951922',
 'view.php?id=freeboard&page=1&

In [31]:
contentList[0]

'view.php?id=freeboard&page=1&divpage=1111&no=5951939'

In [33]:
seed = "http://www.ppomppu.co.kr/zboard/"

In [34]:
seed+contentList[0]

'http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1111&no=5951939'

In [ ]:
title = getURL(seed+contentList[0], params, "td.han")

In [35]:
def getContent(url, params=None, select='a', num_retries=2):
    html = requests.get(url, params=params, headers=headers,)
    
    if 500 <= html.status_code < 600 and nun_retries > 0 :
        print(html.status_code, html.reason)
        return getURL(url, params, num_retries - 1)
    
    #html = html.content.decode('euc-kr').encode('utf-8').decode('utf-8')
    dom = BeautifulSoup(html.text,"html.parser")
    links = dom.select(select) #css selector는 구글에서만 실행
    return links

In [38]:
seed = "http://www.ppomppu.co.kr/zboard/"

for url in contentList:
    title = getContent(seed+url,params,".view_title2")
    print(title[0].get_text())

갓재명 경기다이묘 "정치권, 촛불국민 열망..지방분권 개헌 나서주길"
개고기반대보다  불법교배농장이 금지시키는게 시급하지 않나요?
7700원짜리 옷을 샀는데
삼계탕 먹기 실패했어요
[ 펌- 수정 ] 현재 대한민국 경제 상황 ( 한계기업 중심으로 )
의외로 중진국들이 출산율이 높은가보더라구요
밥먹다가 컬처랜드 보고 같이 먹던 회사사람한테까지 다 말했는데
주휴수당을 폐지해야지만 영세중소기업들이 살아남을 수 있습니다.
삼계탕집 줄서서먹네요
이병철 트윗 - 이재명이 명찰을 억지로 달라는 건
금하고 값어치가 같은 벌레
최영애 "文, 페미니스트 대통령 공약 지켜야"→靑 여권 '수장' 낙점
컬쳐사기가 한바탕지나갔나보네요
Cafe25 먹을만한가요?
주말에 주문한 스케쳐스 ... 헬스장에서 신기에 어떤가요??
주휴수당 없애고 그걸 최저임금에 반영시켜야죠
알라딘 19주년 기념 그림자램프
신종 랜섬웨어
어제나 오늘이나
쉬운한글 쓰기
붓싼이 날씨가 좋긴 하죠.
저는 요새 잠들때 이렇게 합니다
UFC 페더급 공식랭킹 변동  최두호 랭킹 아웃
늦음 점심을 먹으려고 합니다.
김경수가 경남 도지사 출마를 정면돌파한 이유.jpg
중고나라 내가 쓴 글 다 삭제됐네요
아파트 이제 개별난방 전환하는데 대성으로 한다네요
출산율은 여유가 되면 높지 않나요
 자라니.gif (혐, 데이터주의)
